In [ ]:
# Check for ordinary distribution

import pandas as pd
import os
from scipy.stats import shapiro
import numpy as np

# Import file
desktop_path = os.path.expanduser("~/Documents/Ausbildung/Studium/DSHS - SGP/Module/Bachelorarbeit/Ergebnisse")
diagram_path = os.path.join(desktop_path, "Uebersicht.xlsx")

metrics = ["Tore Poisson Bundesliga", 
           "Tore Poisson La Liga",
           "Tore Poisson Ligue 1",
           "Tore Poisson Premier League",
           "Tore Poisson Serie A",
           "Schüsse Bundesliga",
           "Schüsse La Liga",
           "Schüsse Ligue 1",
           "Schüsse Premier League",
           "Schüsse Serie A",
           "VAEP Bundesliga",
           "VAEP La Liga",
           "VAEP Ligue 1",
           "VAEP Premier League",
           "VAEP Serie A",
           "xG Bundesliga",
           "xG La Liga",
           "xG Ligue 1",
           "xG Premier League",
           "xG Serie A",
           "xT Bundesliga",
           "xT La Liga",
           "xT Ligue 1",
           "xT Premier League",
           "xT Serie A"]

# Iterate through each metric and perform the Shapiro-Wilk test
for metric in metrics:
    # Load the data for the current metric (sheet with the name of the metric)
    try:
        df = pd.read_excel(diagram_path, sheet_name=metric)
        
        # Extract the second column (assuming data starts from the second column)
        data = df.iloc[:, 1].dropna().values  # Remove NaN values
        print(data)
        if len(data) >= 3:  # Ensure there are enough data points
            # Perform the Shapiro-Wilk test
            stat, p = shapiro(data)
            
            # Display results
            print(f"Shapiro-Wilk Test for {metric}:")
            print(f"Test statistic: {stat}, p-value: {p}")
            
            # Interpret results
            if p > 0.05:
                print(f"  - Data for {metric} is normally distributed.\n")
            else:
                print(f"  - Data for {metric} is not normally distributed.\n")
        else:
            print(f"Not enough data for {metric}. Skipping Shapiro-Wilk test.\n")
    
    except Exception as e:
        print(f"Error processing {metric}: {e}\n")


In [ ]:
# Check metrics within leagues with Friedman's Test 

import pandas as pd
import os
import numpy as np
import scikit_posthocs as sp
from scipy.stats import friedmanchisquare

# Import file
desktop_path = os.path.expanduser("~/Documents/Ausbildung/Studium/DSHS - SGP/Module/Bachelorarbeit/Ergebnisse")
diagram_path = os.path.join(desktop_path, "Uebersicht.xlsx")

# Load the sheet "Bundesliga gesamt"
sheet_name = "Bundesliga Gesamt"
df = pd.read_excel(diagram_path, sheet_name=sheet_name)

# Define metric names and corresponding columns
metrics = ["Tore", "Schüsse", "VAEP", "xG", "xT"]
columns = [0, 1, 2, 3, 4]  # Column indexes 

# Extract data, ensuring we start from row 2 (excluding header)
data = df.iloc[0:, columns].dropna()  # Drop rows with missing values

# Convert to NumPy array
data_array = data.to_numpy()

print("Data Shape:", data.shape)
print("First few rows of data:\n", data.head())

# Perform the Friedman test
if len(data) >= 3:  # Friedman test requires at least 3 observations
    stat, p = friedmanchisquare(*[data.iloc[:, i].values for i in range(len(metrics))])

    print(f"Friedman Test Results:")
    print(f"Test statistic: {stat}, p-value: {p}")

    if p < 0.05:
        print("There is a significant difference between the metrics.")

        import scipy.stats as stats
        import scikit_posthocs as sp

        # Convert data to long format for Dunn’s test
        long_data = pd.melt(data.reset_index(), id_vars=["index"], value_vars=metrics, var_name="Metric", value_name="Value")
    
        # Perform Dunn's test with Bonferroni correction
        dunn_results = sp.posthoc_dunn(long_data, val_col="Value", group_col="Metric", p_adjust="bonferroni")

        print("\nPairwise Dunn’s Test Results (p-values):")
        print(dunn_results)

        # Ranking the metrics (higher rank = better performance)
        avg_ranks = data.rank(axis=1, method="average").mean()
        best_metric = avg_ranks.idxmax()
        
        print("\nAverage Ranks of Metrics:")
        print(avg_ranks)
        print(f"\nThe best metric based on rankings: {best_metric} ")

    else:
        print("No significant difference between the metrics.")

else:
    print("Not enough data to perform Friedman and Nemenyi test.")


# Export results
desktop_path = os.path.expanduser("~/Desktop")
export_path = os.path.join(desktop_path, "Dunns_Test_Results.xlsx")

# Save results to Excel
with pd.ExcelWriter(export_path) as writer:
    dunn_results.to_excel(writer, sheet_name="Dunns Test P-Values")  # Export p-values
    avg_ranks.to_frame(name="Average Ranks").to_excel(writer, sheet_name="Metric Rankings")  # Export ranking

print(f"Results saved successfully to: {export_path}")


In [ ]:
# Check differences across leagues with Kruskal-Wallis-Test

from scipy.stats import kruskal

# Import file
desktop_path = os.path.expanduser("~/Documents/Ausbildung/Studium/DSHS - SGP/Module/Bachelorarbeit/Ergebnisse")
diagram_path = os.path.join(desktop_path, "Uebersicht.xlsx")

# Load the sheet "Serie A Gesamt"
sheet_name = "Gesamt gesamt"
df = pd.read_excel(diagram_path, sheet_name=sheet_name)

# Define metric names and corresponding columns
metrics = ["Tore", "Schüsse", "VAEP", "xG", "xT"]
columns = [0, 1, 2, 3, 4]  # Column indexes 

# Extract data, ensuring we start from row 2 (excluding header
data = df.iloc[0:, columns].dropna()  # Drop rows with missing values

# Convert to NumPy array
data_array = data.to_numpy()

print("Data Shape:", data.shape)
print("First few rows of data:\n", data.head())

# Perform the Kruskal-Wallis test
if len(data) >= 3:  # Kruskal-Wallis test requires at least 3 observations
    # Applying Kruskal-Wallis test across all metrics
    stat, p = kruskal(*[data.iloc[:, i].values for i in range(len(metrics))])

    print(f"Kruskal-Wallis Test Results:")
    print(f"Test statistic: {stat}, p-value: {p}")

    if p < 0.05:
        print("There is a significant difference between the metrics. Running post-hoc Dunn's test...")

        # Convert data to long format for Dunn’s test
        long_data = pd.melt(data.reset_index(), id_vars=["index"], value_vars=metrics, var_name="Metric", value_name="Value")
    
        # Perform Dunn's test with Bonferroni correction
        dunn_results = sp.posthoc_dunn(long_data, val_col="Value", group_col="Metric", p_adjust="bonferroni")

        print("\nPairwise Dunn’s Test Results (p-values):")
        print(dunn_results)

        # Ranking the metrics (higher rank = better performance)
        avg_ranks = data.rank(axis=1, method="average").mean()
        best_metric = avg_ranks.idxmin()  # Best metric is the one with the lowest rank

        print("\nAverage Ranks of Metrics:")
        print(avg_ranks)
        print(f"\n The best metric based on rankings: {best_metric}")

    else:
        print("No significant difference between the metrics.")

else:
    print("Not enough data to perform Kruskal-Wallis and Dunn's test.")

# Export results
desktop_path = os.path.expanduser("~/Desktop")
export_path = os.path.join(desktop_path, "Dunns_Test_Results_Kruskal_Wallis.xlsx")

# Save results to Excel
with pd.ExcelWriter(export_path) as writer:
    dunn_results.to_excel(writer, sheet_name="Dunns Test P-Values")  # Export p-values
    avg_ranks.to_frame(name="Average Ranks").to_excel(writer, sheet_name="Metric Rankings")  # Export ranking

print(f"Results saved successfully to: {export_path}")